In [1]:
# 02_train_model.ipynb

# 📌 1️⃣ Imports et config
import pandas as pd
from pathlib import Path

from lightgbm import LGBMRegressor, early_stopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score

import mlflow
from mlflow import MlflowClient
from mlflow.exceptions import MlflowException

In [2]:
# 📌 2️⃣ Paramètres
EXPERIMENT_NAME = "ETF_PEA_MLOpsZoomcamp"
mlflow.set_tracking_uri("file:///G:/Mon Drive/DataTalksClub/MLOps-ETF-PEA/mlruns")
mlflow.set_experiment(EXPERIMENT_NAME)

2025/07/15 19:51:24 INFO mlflow.tracking.fluent: Experiment with name 'ETF_PEA_MLOpsZoomcamp' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///G:/Mon Drive/DataTalksClub/MLOps-ETF-PEA/mlruns/516699187525708869', creation_time=1752601884705, experiment_id='516699187525708869', last_update_time=1752601884705, lifecycle_stage='active', name='ETF_PEA_MLOpsZoomcamp', tags={}>

In [3]:
# 📌 3️⃣ Chargement des données
df = pd.read_parquet("../data/df_all.parquet")
print(f"✅ Data shape: {df.shape}")

feature_cols = [col for col in df.columns if col not in ["target", "ticker"]]
X = df[feature_cols]
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

✅ Data shape: (3802, 32)


In [4]:
# 📌 4️⃣ Gestion du Registered Model
client = MlflowClient()
try:
    client.create_registered_model(EXPERIMENT_NAME)
    print(f"✅ Registered model '{EXPERIMENT_NAME}' created.")
except MlflowException as e:
    if "already exists" in str(e):
        print(f"✅ Registered model '{EXPERIMENT_NAME}' already exists, continuing.")
    else:
        raise e

✅ Registered model 'ETF_PEA_MLOpsZoomcamp' created.


In [5]:
# 📌 5️⃣ Tracking et training
params = {
    "learning_rate": 0.01,
    "num_leaves": 31,
    "max_depth": -1,
    "random_state": 42,
}

with mlflow.start_run() as run:
    mlflow.log_params(params)

    model = LGBMRegressor(**params, n_estimators=100)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        callbacks=[early_stopping(stopping_rounds=10)],
    )

    y_pred = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    print(f"✅ Training completed with RMSE: {rmse:.5f}, R²: {r2:.5f}")

    # Log du modèle
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name=EXPERIMENT_NAME,
        input_example=X_test.iloc[:5],
    )

    print(f"✅ Model logged and registered under '{EXPERIMENT_NAME}'")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000361 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6689
[LightGBM] [Info] Number of data points in the train set: 3041, number of used features: 30
[LightGBM] [Info] Start training from score 0.012837
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	training's l2: 0.000988161	valid_1's l2: 0.00116558


2025/07/15 19:53:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Training completed with RMSE: 0.03414, R²: 0.54333
✅ Model logged and registered under 'ETF_PEA_MLOpsZoomcamp'


Registered model 'ETF_PEA_MLOpsZoomcamp' already exists. Creating a new version of this model...
Created version '1' of model 'ETF_PEA_MLOpsZoomcamp'.
